In [3]:
import pandas as pd
df  = pd.read_csv("SampleDataset.csv", encoding='latin-1')

In [4]:
df.head()

,Sno,TECHNOLOGY,ORIGINAL/SYNONYM,QUESTION,SOLUTION
0,1,PowerBI,Original,Issue with Power bi date slicer,If you are experiencing an issue with the date...
1,2,PowerBI,Synonym,What are some common issues that can arise wit...,same as above
2,3,PowerBI,Synonym,How do I resolve issues with the date slicer i...,Same as above
3,4,PowerBI,Synonym,I'm having trouble getting the date slicer in ...,Same as above
4,5,PowerBI,Synonym,What could be causing the date slicer in Power...,Same as above


In [5]:
# I only need column 2nd and 4th
df = df.iloc[:,[1,3]]

In [6]:
df.head()

,TECHNOLOGY,QUESTION
0,PowerBI,Issue with Power bi date slicer
1,PowerBI,What are some common issues that can arise wit...
2,PowerBI,How do I resolve issues with the date slicer i...
3,PowerBI,I'm having trouble getting the date slicer in ...
4,PowerBI,What could be causing the date slicer in Power...
